In [1]:
import pandas as pd
pd.set_option("display.max_row", None)
%load_ext autoreload
%autoreload 2

In [2]:
myAsset = ["Mortgage"
            ,{"originBalance":2600,"originRate":["fix",0.045],"originTerm":30
              ,"freq":"Monthly","type":"Level","originDate":"2021-02-01"}
              ,{"currentBalance":2600
              ,"currentRate":0.08
              ,"remainTerm":30
              ,"status":"current"}]

In [11]:
deal_data = {
    "name":"Multiple Pools"
    ,"dates":{"cutoff":"2021-06-01"
              ,"closing":"2021-07-15"
              ,"firstPay":"2021-08-26"
              ,"payFreq":["DayOfMonth",20]
              ,"poolFreq":"MonthEnd"
              ,"stated":"2030-01-01"}
    ,"pool":{'assets':[myAsset]
            }
    ,"accounts":{"acc01":{"balance":0}
                ,"acc02":{"balance":0}}
    ,"bonds":{"A1":{"balance":1000
                 ,"rate":0.07
                 ,"originBalance":1000
                 ,"originRate":0.07
                 ,"startDate":"2021-07-15"
                 ,"rateType":{"Fixed":0.08}
                 ,"bondType":{"Sequential":None}}
             ,"B":{"balance":1000
                     ,"rate":0.0
                     ,"originBalance":1000
                     ,"originRate":0.07
                     ,"startDate":"2021-07-15"
                     ,"rateType":{"Fixed":0.00}
                     ,"bondType":{"Equity":None}}}
    ,"fees":{"trusteeFee":{"type":{"fixFee":30}}
            ,"serviceFee":{"type":{"annualPctFee":[("poolBalance","PoolB"),0.02]}}
            ,"serviceFee2":{"type":{"byTable":["MonthEnd"
                                               ,("const",1)
                                               ,[(0,5),(2,10),(10,15)]]
                                   }}}
    ,"collect":[["CollectedCash","acc01"]]
    ,"waterfall":{"Amortizing":[
         ["payFee","acc01",['trusteeFee']]
         ,["calcAndPayFee","acc01",['serviceFee']]
         ,["accrueAndPayInt","acc01",["A1"]]
         ,["payPrin","acc01",["A1"]]
         ,["payPrin","acc01",["B"]]
         ,["payPrinResidual","acc01",["B"]]
     ]}
    ,"status":("PreClosing","Amortizing")
}

In [12]:
from absbox import mkDeal,API
deal = mkDeal(deal_data)  ## now a generic class created

In [13]:
localAPI = API("http://localhost:8081")

✅Connected, local lib:0.25.0, server:0.25.0

In [48]:
r = localAPI.run(deal
                 ,poolAssump = ("Pool",
                                  ("Mortgage",{"CDR":0.02} ,None, None, None)
                                  ,None
                                  ,None)
                 ,runAssump = [("interest",("LIBOR6M",0.04))
                              ,("makeWhole"
                                ,"2022-04-20"
                                ,0.001
                                ,[[0.08,0.005],[0.55,0.01],[100,0.02]])]
                 ,read=True)

Warning Message from server:
Bond B is not paid off
Fee serviceFee2 is not paid off

In [34]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-08-26,847.67,8.05,152.33,0.07,160.38,"[<PayInt:A1>, <PayPrin:A1>]"
2021-09-20,756.78,4.06,90.89,0.07,94.95,"[<PayInt:A1>, <PayPrin:A1>]"
2021-10-20,666.34,4.35,90.44,0.07,94.79,"[<PayInt:A1>, <PayPrin:A1>]"
2021-11-20,575.67,3.96,90.67,0.07,94.63,"[<PayInt:A1>, <PayPrin:A1>]"
2021-12-20,484.51,3.31,91.16,0.07,94.47,"[<PayInt:A1>, <PayPrin:A1>]"
2022-01-20,393.08,2.88,91.43,0.07,94.31,"[<PayInt:A1>, <PayPrin:A1>]"
2022-02-20,301.26,2.33,91.82,0.07,94.15,"[<PayInt:A1>, <PayPrin:A1>]"
2022-03-20,208.88,1.61,92.38,0.07,93.99,"[<PayInt:A1>, <PayPrin:A1>]"
2022-04-20,0.00,1.98,208.88,0.07,210.86,"[[[<PayInt:A1>, <PayPrin:A1>], <PayPrin:A1>], ..."


In [54]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-08-26,847.67,8.05,152.33,0.07,160.38,"[<PayInt:A1>, <PayPrin:A1>]"
2021-09-20,756.78,4.06,90.89,0.07,94.95,"[<PayInt:A1>, <PayPrin:A1>]"
2021-10-20,666.34,4.35,90.44,0.07,94.79,"[<PayInt:A1>, <PayPrin:A1>]"
2021-11-20,575.67,3.96,90.67,0.07,94.63,"[<PayInt:A1>, <PayPrin:A1>]"
2021-12-20,484.51,3.31,91.16,0.07,94.47,"[<PayInt:A1>, <PayPrin:A1>]"
2022-01-20,0.00,8.47,484.51,0.07,492.98,"[[[<PayInt:A1>, <PayPrin:A1>], <PayPrin:A1>], ..."


In [52]:
r['bonds']['B']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-08-26,1000.00,0,0.00,0,0.00,"[<PayPrin:B>, <PayPrin:B>]"
2021-09-20,1000.00,0,0.00,0,0.00,"[<PayPrin:B>, <PayPrin:B>]"
2021-10-20,1000.00,0,0.00,0,0.00,"[<PayPrin:B>, <PayPrin:B>]"
2021-11-20,1000.00,0,0.00,0,0.00,"[<PayPrin:B>, <PayPrin:B>]"
2021-12-20,1000.00,0,0.00,0,0.00,"[<PayPrin:B>, <PayPrin:B>]"
2022-01-20,1000.00,0,0.00,0,0.00,"[<PayPrin:B>, <PayPrin:B>]"
2022-02-20,1000.00,0,0.00,0,0.00,"[<PayPrin:B>, <PayPrin:B>]"
2022-03-20,1000.00,0,0.00,0,0.00,"[<PayPrin:B>, <PayPrin:B>]"
2022-04-20,1000.00,0,0.00,0,0.00,"[<PayPrin:B>, <PayPrin:B>]"


In [38]:
r['bonds']['A1']

,balance,interest,principal,rate,cash,memo
date,,,,,,
2021-08-26,1000.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2021-09-20,1000.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2021-10-20,1000.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2021-11-20,1000.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2021-12-20,1000.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2022-01-20,1000.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2022-02-20,1000.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2022-03-20,1000.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
2022-04-20,1000.00,0.00,0.00,0.07,0.00,"[<PayInt:A1>, <PayPrin:A1>]"
